In [4]:
from pathlib import Path
import sys

ROOT = Path().resolve().parents[1] / "code"
sys.path.append(str(ROOT))
import os
import polars as pl
from functions.read_news import read_news
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import pickle


In [5]:
def build_news_embeddings(news_path,save_dir,model_path='/home/ming/GraduateProject/model/all-mpnet-base-v2',batch_size=64):
    os.makedirs(save_dir, exist_ok=True)
    if news_path:
        news=pl.read_parquet(news_path).select(
            ['news_id','title','abstract']
        )
    else:
        news=read_news().select(['news_id','title','abstract','category','subcategory'])
    # texts=(
    #     news['title'].fill_null("")+" "+news['category'].fill_null("")+" "+news['subcategory'].fill_null("")+" "+news['abstract'].fill_null("")
    # ).to_list()

    texts = news['title'].fill_null("").to_list()
    # texts = news['abstract'].fill_null("").to_list()

    model=SentenceTransformer(model_path,device="cuda")

    emb=model.encode(texts,batch_size=batch_size,show_progress_bar=True,normalize_embeddings=True)

    article_ids=news['news_id'].to_list()

    np.save(os.path.join(save_dir,'news_emb.npy'),emb)
    with open(os.path.join(save_dir,'news_ids.pkl'),'wb') as f:
        pickle.dump(article_ids,f)
    print("Saved news embeddings:", emb.shape)

In [6]:
build_news_embeddings(
    news_path='/home/ming/GraduateProject/Data/news.parquet',
    save_dir="/home/ming/GraduateProject/Data/content_emb"
)

Batches: 100%|██████████| 1628/1628 [00:28<00:00, 57.43it/s]


Saved news embeddings: (104151, 768)
